In [1]:
from src.core import *
from src.exps import *
from src.io import *
import altair as alt
from altair_saver import save

In [2]:
CLUSTERING_TARGET_DIR = Path(".").absolute() / "clustering"

In [3]:
all_contexts_names, all_contexts = get_filtered_clusters()
context_overview = pd.read_csv(CONTEXT_NAMES_FP)
context_overview = context_overview[context_overview['mc_id'].isin([int(n) for n in all_contexts_names])]
context_overview['# songs'] = [len(all_contexts['000000{}'.format(c)]) for c in context_overview['mc_id']]
context_overview = context_overview[context_overview['# songs'] > 100]
context_overview.reset_index(inplace=True, drop=True)
context_overview['# subcontexts'] = [get_subcontext_count('000000{}'.format(c)) 
                                     for c in context_overview.mc_id]
context_overview.head()

,mc_id,title,music_performance_rights_id,last_changed,deleted,# songs,# subcontexts
0,8001,Opgewekt,1,NaN,NaN,2587,1
1,8002,Pure Blues,1,NaN,NaN,1088,1
2,8003,Rhythm & Blues,1,NaN,NaN,1129,1
3,8007,Dynamisch,1,NaN,NaN,872,2
4,8009,Pop & Blues,1,NaN,NaN,329,1


In [342]:
context_overview[context_overview['# songs'] > 5000]

,mc_id,title,music_performance_rights_id,last_changed,deleted,# songs,# subcontexts
19,8073,Gezellig,1,NaN,NaN,6204,1
20,8074,Opgewekt,1,NaN,NaN,12028,1
58,8185,Rock,1,NaN,NaN,12918,1
62,8204,Soul & Funk,1,NaN,NaN,5019,1
66,8218,Old Skool,1,NaN,NaN,5163,1
71,8266,Duits,1,NaN,NaN,7328,1
72,8268,Italiaans,1,NaN,NaN,5216,1
89,8432,Dynamic,1,NaN,NaN,6866,1
101,8455,Country Lite,1,NaN,NaN,5220,2
113,8483,Pop Rock,1,NaN,NaN,6120,1


In [343]:
with open(CLUSTERING_TARGET_DIR/ 'results_clustering_0906.txt','r') as f:
    results_clustering = f.readlines()

In [344]:
results_clustering = [r[:-1].split(',') for r in results_clustering]

In [345]:
len(results_clustering)

72154

In [346]:
results_clustering_processed = []
for r in results_clustering:
    processed = [float(r[i]) if i not in [3, 7] else r[i][1:] for i in range(len(r))]
    results_clustering_processed.append(processed)
results_clustering_processed[0]

[5.0, 1.0, 8722.0, 'l', 0.1, 0.0, 926.0, 'dt-query', 1068.0, 0.0, 0.0, nan]

In [347]:
results_clustering_processed = pd.DataFrame(results_clustering_processed)
results_clustering_processed.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,5.0,1.0,8722.0,l,0.1,0.0,926.0,dt-query,1068.0,0.0,0.0,NaN
1,5.0,1.0,8722.0,l,0.1,0.0,926.0,songs-query,1039.0,0.0,0.0,NaN
2,5.0,1.0,8074.0,l,0.1,0.0,12028.0,dt-query,1068.0,0.0,0.0,NaN
3,5.0,1.0,8074.0,l,0.1,0.0,12028.0,songs-query,1039.0,0.0,0.0,NaN
4,5.0,1.0,8726.0,l,0.1,0.0,832.0,dt-query,1068.0,0.0,0.0,NaN


In [348]:
results_clustering_processed.columns = ['# experiment', '# learned context', 'source context', 'reliable negative method',
                              'discard threshold', 'rocchio threshold', '# songs target', 'query type', '# songs predicted', 
                              'recall', 'precision', 'f1-score']
results_clustering_processed = results_clustering_processed[results_clustering_processed['rocchio threshold'] == 0]
results_clustering_processed.reset_index(inplace=True, drop=True)
results_clustering_processed.shape

(72154, 12)

In [355]:
for i in range(1, 11):
    contexts = len(np.unique(results_clustering_processed[(results_clustering_processed['# experiment'] == i)]['# learned context']))
    for j in range(1, contexts + 1):
        sources = np.unique(results_clustering_processed[(results_clustering_processed['# experiment'] == i) & 
                                     (results_clustering_processed['# learned context'] == j)]['source context'])
        if len(sources) < 40:
            print(i, j, len(sources))

9 40 10


In [356]:
overview = pd.DataFrame()
for n in set(results_clustering_processed['# experiment']):
    for e in set(results_clustering_processed[(results_clustering_processed['# experiment'] == n)]['# learned context']):
        for reliable_negative_method in ['r', 'l']:
            for t in [0, 0.1, 0.2]:
                for q in ['dt-query', 'songs-query']:
                    subset = results_clustering_processed[(results_clustering_processed['# experiment'] == n) & 
                                                          (results_clustering_processed['# learned context'] == e) & 
                                                          (results_clustering_processed['reliable negative method'] == reliable_negative_method) &
                                                          (results_clustering_processed['discard threshold'] == t) & 
                                                          (results_clustering_processed['query type'] == q)]
                    subset.reset_index(inplace=True, drop=True)
                    if subset.shape[0] > 0:
                        max_index = subset["f1-score"].idxmax()
                        if math.isnan(max_index):
                            subset = subset.iloc[0, :]
                        else:
                            subset = subset.iloc[max_index, :]
                        overview = overview.append(subset)
overview.reset_index(inplace=True, drop=True)

In [357]:
overview.shape, 10*40*40*6*2

((1816, 12), 192000)

In [367]:
overlap_scores = pd.DataFrame()
for threshold in [0, 0.1, 0.2]:
    for q in ['dt-query', 'songs-query']:
        for r in ['r', 'l']:
            data = overview[(overview['discard threshold'] == threshold) & 
                            (overview['query type'] == q) &
                            (overview['reliable negative method'] == r)]
            s = data[data['f1-score'] >= 0.7].shape[0]/data.shape[0]
            overlap_scores = overlap_scores.append([[q, r, threshold, data.shape[0], s]])

overlap_scores.columns = ['query type', 'reliable negative approach', 'discard threshold', '# learned contexts', 'score']
overlap_scores

,query type,reliable negative approach,discard threshold,# learned contexts,score
0,dt-query,r,0.0,151,0.774834
0,dt-query,l,0.0,160,0.862500
0,songs-query,r,0.0,151,0.847682
0,songs-query,l,0.0,160,0.812500
0,dt-query,r,0.1,78,0.769231
0,dt-query,l,0.1,142,0.739437
0,songs-query,r,0.1,78,0.820513
0,songs-query,l,0.1,142,0.732394
0,dt-query,r,0.2,160,0.762500
0,dt-query,l,0.2,217,0.686636


In [ ]:
def big_chart(chart, fontsize = 20):
    return chart.configure_axis(
        grid = True,
    labelFontSize = fontsize,
    titleFontSize = fontsize
).configure_title(
    fontSize = fontsize
    ).configure_legend(
titleFontSize=fontsize,
labelFontSize=fontsize
).configure_view(
    strokeWidth=0
)

def small_chart(chart, fontsize=None):
    return big_chart(chart.properties(width=150,
                             height=150
                            ), fontsize)

In [370]:
data = overlap_scores[overlap_scores['query type'] == 'dt-query']
chart_d = alt.Chart(data).mark_bar().encode(
    alt.X("reliable negative approach", title=None),
    alt.Y('score', title='overlap accuracy'),
    alt.Color('reliable negative approach', scale=alt.Scale(scheme='tableau10'), title = 'Rel. Neg.'), 
    alt.Column('discard threshold', header=alt.Header(titleFontSize=20, labelFontSize=20))
)
big_chart(chart_d, 20)

alt.Chart(...)

In [371]:
data = overlap_scores[overlap_scores['query type'] == 'songs-query']
chart_s = alt.Chart(data).mark_bar().encode(
    alt.X("reliable negative approach", title=None),
    alt.Y('score', title='overlap accuracy'),
    alt.Color('reliable negative approach', scale=alt.Scale(scheme='tableau10'), title = 'Rel. Neg.'), 
    alt.Column('discard threshold', header=alt.Header(titleFontSize=20, labelFontSize=20))
)
big_chart(chart_s, 20)


alt.Chart(...)

In [212]:
PLOT_DIR = str(Path().absolute() / 'plots-clustering')

for threshold in [0, 0.1, 0.2]:
    for q in ['dt-query', 'songs-query']:
        for r in ['r', 'l']:
            data = overview[(overview['discard threshold'] == threshold) & 
                            (overview['query type'] == q) &
                            (overview['reliable negative method'] == r)]
            if data.shape[0] > 0:
                chart_hist = alt.Chart(data).mark_bar().encode(
                                                        alt.X("f1-score:Q", bin=True),
                                                        alt.Y('count()', title='Count')
                                                    )
                save(big_chart(chart_hist, fontsize=20), '{}/f1_score_query_{}_discard_{}_rel_{}.png'.format(PLOT_DIR, q, threshold, r), scale_factor=2.0)
        
    
